In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import WordPunctTokenizer
import gensim
from gensim.models import KeyedVectors
from IPython.display import clear_output
from collections import Counter
from tqdm import tqdm

%matplotlib inline

In [83]:
%run utils.py

In [38]:
path = 'data/requests.uniq.train'
train = read_queries(path)
path = 'data/requests.uniq.test'
test = read_queries(path)
train[:5], test[:5]

([['сибирские', 'сети', 'личный', 'кабинет', 'бердск'],
  ['1', 'сантим', 'алжир', '1964'],
  ['река', 'колыма', 'на', 'карте', 'россии'],
  ['ноофен', 'для', 'каких', 'болезней'],
  ['маус', 'хаус', 'спб']],
 [['сбербанк', 'в', 'кунцево', 'плаза'],
  ['торт', 'дикая', 'вишня'],
  ['тася', 'кривун', 'танцы', 'на', 'тнт'],
  ['рбт', 'ру'],
  ['toplü', 'vay', 'sexx']])

### LSTM

In [5]:
emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [40]:
emb_size = 300

train_part, valid_part = split_to_train_and_validation(train)
train_prefixs, train_next_words = transform(train_part)
valid_prefixs, valid_next_words = transform(valid_part)
print(len(train_prefixs), len(valid_prefixs))

137449 34145


In [63]:
train_prefixs = np.array(train_prefixs)
train_next_words = np.array(train_next_words)
valid_prefixs = np.array(valid_prefixs)
valid_next_words = np.array(valid_next_words)

In [41]:
import keras
import keras.layers as L

Using TensorFlow backend.


In [64]:
def build_model(emb_size=300, hid_size=64):
    
    l_input = L.Input(shape=[None, emb_size], name='input')
    l_lstm = L.LSTM(units=hid_size)(l_input)
    
    output_layer = L.Dense(emb_size)(l_lstm)
    
    model = keras.models.Model(inputs=[l_input], outputs=[output_layer])
    model.compile('adam', 'mean_squared_error', metrics=['mean_absolute_error'])
    return model
    
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, None, 300)         0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                93440     
_________________________________________________________________
dense_14 (Dense)             (None, 300)               19500     
Total params: 112,940
Trainable params: 112,940
Non-trainable params: 0
_________________________________________________________________


In [73]:
batch_size = 256
epochs = 10            
steps_per_epoch = 100  

model = build_model()

model.fit_generator(iterate_minibatches(emb, emb_size, train_prefixs, train_next_words, batch_size, cycle=True), 
                    epochs=epochs, steps_per_epoch=steps_per_epoch,
                    
                    validation_data=iterate_minibatches(emb, emb_size, valid_prefixs, valid_next_words, batch_size, cycle=True),
                    validation_steps=len(valid_prefixs) // batch_size
                   )

Epoch 1/10
100/100 [==============================] - 18s 184ms/step - loss: 0.0100 - mean_absolute_error: 0.0594 - val_loss: 0.0096 - val_mean_absolute_error: 0.0585
Epoch 2/10
100/100 [==============================] - 14s 136ms/step - loss: 0.0096 - mean_absolute_error: 0.0585 - val_loss: 0.0095 - val_mean_absolute_error: 0.0585
Epoch 3/10
100/100 [==============================] - 14s 142ms/step - loss: 0.0095 - mean_absolute_error: 0.0584 - val_loss: 0.0094 - val_mean_absolute_error: 0.0579
Epoch 4/10
100/100 [==============================] - 14s 140ms/step - loss: 0.0094 - mean_absolute_error: 0.0581 - val_loss: 0.0093 - val_mean_absolute_error: 0.0578
Epoch 5/10
100/100 [==============================] - 16s 156ms/step - loss: 0.0094 - mean_absolute_error: 0.0580 - val_loss: 0.0093 - val_mean_absolute_error: 0.0575
Epoch 6/10
100/100 [==============================] - 15s 145ms/step - loss: 0.0094 - mean_absolute_error: 0.0582 - val_loss: 0.0093 - val_mean_absolute_error: 0.057

In [74]:
test_prefixs, test_next_words = transform(test)
test_prefixs = np.array(test_prefixs)
test_next_words = np.array(test_next_words)
len(test_prefixs)

70146

In [84]:
calculate_accuracy(model, emb, emb_size, 256, test_prefixs, test_next_words)

In progress 512 / 70146, accuracy = 0.0546875


KeyboardInterrupt: 